In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load


import tensorflow as tf
from sklearn.model_selection import train_test_split 
from sklearn.metrics import classification_report, accuracy_score
from sklearn import preprocessing 
import matplotlib.pyplot as plt
import pandas as pd 
import numpy as np
import pandas as pd 
import os

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### introduction

from idea of https://www.kaggle.com/shivamb/semi-supervised-classification-using-autoencoders
I tried to find latent representation on this topic, and it seemed quite legit for a while... until I found out it makes similar score if we just put 
features into XGboost... 
Also thanks to https://www.kaggle.com/pratikkgandhi/rapids-rf-classifier-on-tps-sept-0-80476-lb,
I used idea of making nan into median value of the rows, and making the 'nan count' features...

it seems not many people used the autoencoder. 
Does anybody knows why? plz let me know...

In [ ]:
path = '/kaggle/input/tabular-playground-series-sep-2021/'
trainpath = os.path.join(path,'train.csv')
testpath = os.path.join(path,'test.csv')
X_test = pd.read_csv(testpath)
testid = X_test['id']
df_train = pd.read_csv(trainpath)

In [ ]:
features = df_train.columns[:-1]
target = df_train.columns[-1]
df_train["Missing_Value"] = df_train[features].isna().sum(axis=1) 
df_train["Missing_Value_cat"] = df_train["Missing_Value"].map(lambda x: 0 if x==0 else 1)
X_test["Missing_Value"] = X_test[features].isna().sum(axis=1) 
X_test["Missing_Value_cat"] = X_test["Missing_Value"].map(lambda x: 0 if x==0 else 1)

In [ ]:
for column in features:
    avg_val = df_train[column].median()
    df_train[column].fillna(avg_val, inplace=True)
    X_test[column].fillna(avg_val, inplace=True)
features = list(features)
features.extend(['Missing_Value','Missing_Value_cat'])

In [ ]:
x = df_train.drop(["claim"], axis=1)
y = df_train["claim"].values

x_scale = preprocessing.MinMaxScaler().fit_transform(x.values)
x_norm, x_claim = x_scale[y == 0], x_scale[y == 1]

In [ ]:
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras import regularizers

In [ ]:
## input layer 
input_layer = Input(shape=(x.shape[1],))

## encoding part
encoded = Dense(512, activation='tanh', activity_regularizer=regularizers.l1(10e-5))(input_layer)
encoded = Dense(256, activation='relu')(encoded)
encoded = Dense(128, activation='relu')(encoded)
encoded = Dense(64, activation='relu')(encoded)

decoded = Dense(64, activation= 'tanh')(encoded)
decoded = Dense(128, activation='tanh')(decoded)
decoded = Dense(256, activation='tanh')(decoded)
decoded = Dense(512, activation='tanh')(decoded)


## output layer
output_layer = Dense(x.shape[1], activation='relu')(decoded)

In [ ]:
es_callback = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    min_delta=0.0001,
    patience=5,
    verbose=0,
    mode="auto",
    baseline=None,
    restore_best_weights=True,
)


In [ ]:
autoencoder = Model(input_layer, output_layer)
autoencoder.compile(optimizer="adam", loss=tf.keras.losses.MeanSquaredLogarithmicError(
    reduction="auto", name="mean_squared_logarithmic_error"
))

In [ ]:
autoencoder.fit(x_norm, x_norm, 
                batch_size = 512, epochs = 30, 
                shuffle = True, validation_split = 0.20,callbacks=[es_callback]);

In [ ]:
hidden_representation = Sequential()
n=5
for i in range(n):
    hidden_representation.add(autoencoder.layers[i])  

In [ ]:
norm_hid_rep = hidden_representation.predict(x_norm)
claim_hid_rep = hidden_representation.predict(x_claim)
rep_x = np.append(norm_hid_rep, claim_hid_rep, axis = 0)
y_n = np.zeros(norm_hid_rep.shape[0])
y_f = np.ones(claim_hid_rep.shape[0])
rep_y = np.append(y_n, y_f)

In [ ]:
from xgboost import XGBClassifier


model = XGBClassifier(
    max_depth=5,
    subsample=0.5,
    colsample_bytree=0.5,
    n_jobs=-1,
    random_state=0,
)

In [ ]:
train_x, val_x, train_y, val_y = train_test_split(rep_x, rep_y, test_size=0.25)
clf = model.fit(train_x, train_y)
pred_y = clf.predict(val_x)

In [ ]:
print ("")
print ("Classification Report: ")
print (classification_report(val_y, pred_y))

print ("")
print ("Accuracy Score: ", accuracy_score(val_y, pred_y))

In [ ]:
X_test = preprocessing.MinMaxScaler().fit_transform(X_test)
X_test = hidden_representation.predict(X_test)

In [ ]:

# Make predictions
y_pred_test = pd.Series(model.predict(X_test))
y_pred_test = pd.DataFrame(y_pred_test,columns=['claim'])
y_pred_test['id'] = testid
y_pred_test = y_pred_test[['id','claim']]
# Create submission file
y_pred_test.to_csv("submission.csv",index=False)

In [ ]:
from collections import Counter
y_counter = Counter(y_pred_test['claim'])
print(y_counter)